<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>


# Formatting Data (dates)

It is very common to find dates (some combination of year, month, day of week and time) in data that is collected in real time (and other that organize event information.

Let's see a data frame that comes with dates from an API.

In [1]:
import pandas as pd
from sodapy import Socrata

client = Socrata("data.seattle.gov", None)

results = client.get("kzjm-xkqj", limit=2000)

# Convert to pandas DataFrame
calls911 = pd.DataFrame.from_records(results)

Let's check some information:

In [2]:
calls911.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   address                      2000 non-null   object
 1   type                         2000 non-null   object
 2   datetime                     2000 non-null   object
 3   latitude                     1999 non-null   object
 4   longitude                    1999 non-null   object
 5   report_location              1999 non-null   object
 6   incident_number              2000 non-null   object
 7   :@computed_region_ru88_fbhk  1991 non-null   object
 8   :@computed_region_kuhn_3gp2  1991 non-null   object
 9   :@computed_region_q256_3sug  1999 non-null   object
 10  :@computed_region_2day_rhn5  151 non-null    object
 11  :@computed_region_cyqu_gs94  141 non-null    object
dtypes: object(12)
memory usage: 187.6+ KB


Let's get rid of some columns:

In [3]:
calls911=calls911.iloc[:,:7]

Let's check the column _datetime_:

In [4]:
calls911.datetime.head()

0    2023-05-04T13:09:00.000
1    2023-05-04T13:03:00.000
2    2023-05-04T13:02:00.000
3    2023-05-04T13:02:00.000
4    2023-05-04T12:52:00.000
Name: datetime, dtype: object

In [5]:
# then
type(calls911.datetime[0])

str

The date and time information is not useful at this time, that is, the information it offers is of limited use, as it is just a string. 

Let's make it useful:

In [6]:
calls911.datetime=pd.to_datetime(calls911.datetime,format='%Y-%m-%d')
calls911.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   address          2000 non-null   object        
 1   type             2000 non-null   object        
 2   datetime         2000 non-null   datetime64[ns]
 3   latitude         1999 non-null   object        
 4   longitude        1999 non-null   object        
 5   report_location  1999 non-null   object        
 6   incident_number  2000 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 109.5+ KB


Once you have this data type, you can retrieve important information:

In [7]:
calls911['date']=calls911.datetime.dt.date
calls911['year']=calls911.datetime.dt.year
calls911['month']=calls911.datetime.dt.month
calls911['month_name']=calls911.datetime.dt.month_name()
calls911['day']=calls911.datetime.dt.day
calls911['weekday']=calls911.datetime.dt.day_name()
calls911['hour']=calls911.datetime.dt.hour
calls911['minute']=calls911.datetime.dt.minute

In [8]:
calls911.head()

,address,type,datetime,latitude,longitude,report_location,incident_number,date,year,month,month_name,day,weekday,hour,minute
0,509 10th Ave E,Aid Response,2023-05-04 13:09:00,47.623218,-122.319906,"{'type': 'Point', 'coordinates': [-122.319906,...",F230053743,2023-05-04,2023,5,May,4,Thursday,13,9
1,1430 2nd Ave,Aid Response,2023-05-04 13:03:00,47.608586,-122.338264,"{'type': 'Point', 'coordinates': [-122.338264,...",F230053742,2023-05-04,2023,5,May,4,Thursday,13,3
2,1510 Nw 52nd St,Medic Response- Overdose,2023-05-04 13:02:00,47.666433,-122.376341,"{'type': 'Point', 'coordinates': [-122.376341,...",F230053740,2023-05-04,2023,5,May,4,Thursday,13,2
3,1105 E Fir St,Medic Response,2023-05-04 13:02:00,47.602688,-122.318034,"{'type': 'Point', 'coordinates': [-122.318034,...",F230053741,2023-05-04,2023,5,May,4,Thursday,13,2
4,4219 S Othello St,Crisis Center,2023-05-04 12:52:00,47.53705,-122.280473,"{'type': 'Point', 'coordinates': [-122.280473,...",F230053738,2023-05-04,2023,5,May,4,Thursday,12,52


Let's create a new column with what we have. In this case, a boolean where you tell if it is night time (after 8 pm before 6 am):

In [9]:
calls911['nightTime']=((calls911['hour']<=6) | (calls911['hour']>=20))

Can we use several columns to build a _datetime_?

In [10]:
pd.to_datetime(calls911[['month', 'day','year','hour','minute']])

0      2023-05-04 13:09:00
1      2023-05-04 13:03:00
2      2023-05-04 13:02:00
3      2023-05-04 13:02:00
4      2023-05-04 12:52:00
               ...        
1995   2023-04-28 19:28:00
1996   2023-04-28 19:28:00
1997   2023-04-28 19:26:00
1998   2023-04-28 19:22:00
1999   2023-04-28 19:20:00
Length: 2000, dtype: datetime64[ns]

Notice that the columns about lat/long are non numeric, let's solve that:

In [11]:
calls911[['longitude','latitude']]=calls911[['longitude','latitude']].apply(lambda x:pd.to_numeric(x))

In [12]:
#re check
calls911.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   address          2000 non-null   object        
 1   type             2000 non-null   object        
 2   datetime         2000 non-null   datetime64[ns]
 3   latitude         1999 non-null   float64       
 4   longitude        1999 non-null   float64       
 5   report_location  1999 non-null   object        
 6   incident_number  2000 non-null   object        
 7   date             2000 non-null   object        
 8   year             2000 non-null   int64         
 9   month            2000 non-null   int64         
 10  month_name       2000 non-null   object        
 11  day              2000 non-null   int64         
 12  weekday          2000 non-null   object        
 13  hour             2000 non-null   int64         
 14  minute           2000 non-null   int64  

Let's save what we have:

In [13]:
import os

where=os.path.join('data','calls911.pkl')
calls911.to_pickle(where)